### Essential Libraries

In [1]:
import torch
import csv
import os
import time
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from dataloader import ATeX  # Ensure dataloader.py is in the same directory or in PYTHONPATH
import torchvision.models as models
from sklearn.metrics import classification_report  # Import the function
from torch.optim import SGD
import numpy as np
from torchvision.models import efficientnet_b0
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingLR

### Dataset Preparation and DataLoader Setup

In [2]:
# Define transformations and dataset
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_std[0], mean_std[1]),
])

train_dataset = ATeX(split="train", transform=data_transforms)
test_dataset = ATeX(split="test", transform=data_transforms)

# Define DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Validation Dataset and DataLoader
val_dataset = ATeX(split="val", transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### Define the CNN model

In [3]:
# Load pre-trained EfficientNet-B0 model and modify for 15 classes
class EffNetB0(nn.Module):
    def __init__(self, num_classes=15):
        super(EffNetB0, self).__init__()
        self.effnet = efficientnet_b0(pretrained=True)
        # Replace the classifier with a new one for 15 classes
        self.effnet.classifier[1] = nn.Linear(self.effnet.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.effnet(x)

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EffNetB0(num_classes=15).to(device)


c:\Users\Roja\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Roja\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training, Validation and Evaluation

In [4]:
# Initialize lists to store metrics
metrics = []

# Address to save the CSV file
model_name = "efficientnet_b0"
save_dir = r"D:/USC_Course/CSCE 790 Section 007 Neural Networks and Their Applications/atex-main/loss_training time/"
save_path = os.path.join(save_dir, f"training_metrics_{model_name}.csv")

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Define optimizer, scheduler, and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
criterion = nn.CrossEntropyLoss()  # Replace with label_smoothing_loss if needed

# Record the start time of the training process
training_start_time = time.time()

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    # Training Phase
    model.train()
    running_loss = 0.0
    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)  # Use the defined loss function

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average training loss
    train_loss = running_loss / len(train_loader)

    # Update learning rate
    scheduler.step()

    # Validation Phase
    model.eval()
    val_loss = 0.0
    all_val_labels = []
    all_val_preds = []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())

    # Calculate validation metrics
    val_loss /= len(val_loader)
    val_report = classification_report(
        all_val_labels, all_val_preds, target_names=train_dataset.classes, zero_division=0, output_dict=True
    )
    val_precision = val_report['weighted avg']['precision']
    val_recall = val_report['weighted avg']['recall']
    val_f1 = val_report['weighted avg']['f1-score']

    # Save metrics for this epoch
    metrics.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "precision": val_precision,
        "recall": val_recall,
        "f1_score": val_f1
    })

    # Print training and validation metrics
    print(f"Epoch [{epoch+1}/{num_epochs}]: Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, "
          f"Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}")

# Record the end time of the training process
training_end_time = time.time()
total_training_time_seconds = training_end_time - training_start_time
total_training_time_minutes = total_training_time_seconds / 60

# Save metrics to a CSV file
with open(save_path, 'w', newline='') as csvfile:
    fieldnames = ["epoch", "train_loss", "val_loss", "precision", "recall", "f1_score"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(metrics)

# Append total training time in minutes
with open(save_path, 'a', newline='') as csvfile:
    csvfile.write(f"\nTotal Training Time (minutes):,{total_training_time_minutes:.2f}\n")

print(f"Training metrics saved to: {save_path}")
print(f"Total training time for all epochs: {total_training_time_minutes:.2f} minutes")


Epoch [1/30]: Train Loss: 2.0943, Val Loss: 1.6475, Precision: 0.4707, Recall: 0.4784, F1-Score: 0.4639
Epoch [2/30]: Train Loss: 1.4549, Val Loss: 1.1589, Precision: 0.6372, Recall: 0.6334, F1-Score: 0.6302
Epoch [3/30]: Train Loss: 1.1241, Val Loss: 0.9293, Precision: 0.7058, Recall: 0.6949, F1-Score: 0.6931
Epoch [4/30]: Train Loss: 0.8866, Val Loss: 0.7617, Precision: 0.7740, Recall: 0.7644, F1-Score: 0.7640
Epoch [5/30]: Train Loss: 0.7514, Val Loss: 0.7017, Precision: 0.7851, Recall: 0.7748, F1-Score: 0.7730
Epoch [6/30]: Train Loss: 0.6078, Val Loss: 0.6482, Precision: 0.7909, Recall: 0.7780, F1-Score: 0.7794
Epoch [7/30]: Train Loss: 0.5164, Val Loss: 0.5586, Precision: 0.8267, Recall: 0.8227, F1-Score: 0.8217
Epoch [8/30]: Train Loss: 0.4495, Val Loss: 0.5934, Precision: 0.8209, Recall: 0.8115, F1-Score: 0.8130
Epoch [9/30]: Train Loss: 0.3911, Val Loss: 0.4955, Precision: 0.8497, Recall: 0.8442, F1-Score: 0.8456
Epoch [10/30]: Train Loss: 0.3228, Val Loss: 0.4570, Precision: 

### Testing and Evaluation

In [5]:
# Testing the model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels, _ in test_loader:  # Assuming your DataLoader provides 3 items
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Calculate classification metrics
class_names = train_dataset.classes  # Assuming train_dataset has the class names
report = classification_report(all_labels, all_preds, target_names=class_names)
print("Classification Report:\n")
print(report)

# Compute per-class accuracy
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)



Classification Report:

              precision    recall  f1-score   support

       delta       0.96      0.97      0.96       330
     estuary       0.91      0.94      0.92       125
       flood       0.88      0.89      0.88       283
    glaciers       0.97      0.96      0.97       240
  hot_spring       0.85      0.90      0.88       201
        lake       0.91      0.88      0.89       153
        pool       0.95      0.94      0.94        79
      puddle       0.83      0.80      0.81       168
      rapids       0.89      0.85      0.87       219
       river       0.74      0.81      0.77        73
         sea       0.90      0.94      0.92       108
        snow       0.96      0.92      0.94       142
       swamp       0.96      0.93      0.94       188
   waterfall       0.84      0.79      0.81        96
     wetland       0.95      0.97      0.96        93

    accuracy                           0.91      2498
   macro avg       0.90      0.90      0.90      2498
we

### Save Results

In [6]:
# Save classification report
with open("classification_report_EfficientNet-B0 model.txt", "w") as f:
    f.write(report)

# Save model weights
torch.save(model.state_dict(), "EfficientNet-B0 model_cpu.pth")